# day 16

https://adventofcode.com/16/day/16

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day16.txt')

LOGGER = logging.getLogger('day16')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|...."""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
def parse_data(s: str) -> dict[complex, str]:
    return {(i + j * 1j): char
            for (i, row) in enumerate(s.strip().split('\n'))
            for (j, char) in enumerate(row)}

In [ ]:
from enum import Enum


class Direction(Enum):
    UP = 'up'
    DOWN = 'down'
    LEFT = 'left'
    RIGHT = 'right'


dir_map = {Direction.UP: -1,
           Direction.DOWN: +1,
           Direction.LEFT: -1j,
           Direction.RIGHT: +1j}


class Contraption:
    def __init__(self, data: str, starting_loc: complex = -1j, starting_dir: Direction = Direction.RIGHT):
        self.data = data
        self.map = parse_data(self.data)
        self.beam_history = set()
        self.beam_locs = {starting_loc: {starting_dir}}
        self.beam_starting_locs = {(starting_loc, starting_dir),}

    @property
    def height(self) -> int:
        return max(int(_.real) for _ in self.map)

    @property
    def width(self) -> int:
        return max(int(_.imag) for _ in self.map)

    def step(self) -> None:
        new_beam_locs = {}
        for beam_loc, beam_directions in self.beam_locs.items():
            for beam_direction in beam_directions:
                new_beam_loc = beam_loc + dir_map[beam_direction]
                if new_beam_loc not in self.map:
                    continue

                if new_beam_loc not in new_beam_locs:
                    new_beam_locs[new_beam_loc] = set()
                for new_beam_direction in self.resolve_beam_dirs(new_space=new_beam_loc,
                                                                 current_dir=beam_direction):
                    new_beam_locs[new_beam_loc].add(new_beam_direction)
                self.beam_history.add(new_beam_loc)
        self.beam_locs = new_beam_locs

    def resolve_beam_dirs(self, new_space: complex, current_dir: Direction) -> list[Direction]:
        match self.map[new_space]:
            case '.':
                return [current_dir]
            case '|':
                match current_dir:
                    case Direction.LEFT | Direction.RIGHT:
                        return [Direction.UP, Direction.DOWN]
                    case _:
                        return [current_dir]
            case '-':
                match current_dir:
                    case Direction.UP | Direction.DOWN:
                        return [Direction.LEFT, Direction.RIGHT]
                    case _:
                        return [current_dir]
            case '\\':
                match current_dir:
                    case Direction.UP:
                        return [Direction.LEFT]
                    case Direction.DOWN:
                        return [Direction.RIGHT]
                    case Direction.LEFT:
                        return [Direction.UP]
                    case Direction.RIGHT:
                        return [Direction.DOWN]
                    case _:
                        raise ValueError(f"unhandled character / current dir = {self.map[new_space], current_dir}")
            case '/':
                match current_dir:
                    case Direction.UP:
                        return [Direction.RIGHT]
                    case Direction.DOWN:
                        return [Direction.LEFT]
                    case Direction.LEFT:
                        return [Direction.DOWN]
                    case Direction.RIGHT:
                        return [Direction.UP]
                    case _:
                        raise ValueError(f"unhandled character / current dir = {self.map[new_space], current_dir}")
            case _:
                raise ValueError(f"Unhandled character {self.map[new_space]}")

    def history_as_str(self) -> str:
        s = ''
        for i in range(self.height + 1):
            for j in range(self.width + 1):
                loc = i + j * 1j
                s += '#' if loc in self.beam_history else '.'
            s += '\n'
        return s

    def beam_locs_as_str(self) -> str:
        s = ''
        for beam_loc in sorted(self.beam_locs.keys(), key=lambda x: (x.real, x.imag)):
            s += f"{beam_loc}:"
            for direction in self.beam_locs[beam_loc]:
                s += f"{direction};"
            s += '|'
        return s

    @property
    def num_energized(self) -> int:
        return self.history_as_str().count('#')

In [ ]:
c = Contraption(test_data)
for i in range(10):
    c.step()
    print(f"after step {i},")
    print(f"  {c.beam_history = }")
    print(f"  {c.beam_locs = }")
    print(f"  {c.beam_locs_as_str() = }")

for i in range(100):
    c.step()
print()
print(c.history_as_str())
print()

print(c.num_energized)

In [ ]:
def q_1(data, num_steps=100):
    c = Contraption(data)
    for i in range(num_steps):
        c.step()

    return c.num_energized

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 46
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data(), 1_000)

## part 2

### problem statement:

#### function def

In [ ]:
import networkx as nx

In [ ]:
def map_to_graph(map: dict[complex: str]) -> nx.DiGraph:
    g = nx.DiGraph()

    for (loc, char) in map.items():
        l_node = loc - 0.5 * 1j
        r_node = loc + 0.5 * 1j
        u_node = loc - 0.5
        d_node = loc + 0.5
        # edges are "enter from -> exit to"
        match char:
            case '.':
                g.add_edge(l_node, r_node, loc=loc)
                g.add_edge(r_node, l_node, loc=loc)
                g.add_edge(u_node, d_node, loc=loc)
                g.add_edge(d_node, u_node, loc=loc)
            case '-':
                g.add_edge(l_node, r_node, loc=loc)
                g.add_edge(r_node, l_node, loc=loc)
                g.add_edge(u_node, l_node, loc=loc)
                g.add_edge(u_node, r_node, loc=loc)
                g.add_edge(d_node, l_node, loc=loc)
                g.add_edge(d_node, r_node, loc=loc)
            case '|':
                g.add_edge(l_node, u_node, loc=loc)
                g.add_edge(l_node, d_node, loc=loc)
                g.add_edge(r_node, u_node, loc=loc)
                g.add_edge(r_node, d_node, loc=loc)
                g.add_edge(u_node, d_node, loc=loc)
                g.add_edge(d_node, u_node, loc=loc)
            case '/':
                g.add_edge(l_node, u_node, loc=loc)
                g.add_edge(u_node, l_node, loc=loc)
                g.add_edge(r_node, d_node, loc=loc)
                g.add_edge(d_node, r_node, loc=loc)
            case '\\':
                g.add_edge(l_node, d_node, loc=loc)
                g.add_edge(d_node, l_node, loc=loc)
                g.add_edge(r_node, u_node, loc=loc)
                g.add_edge(u_node, r_node, loc=loc)
    return g

In [ ]:
map = parse_data(test_data)
g = map_to_graph(map=map)

def draw_graph(g):
    pos = {n: (n.imag, -n.real) for n in g.nodes}
    nx.draw(g, pos=pos, node_size=50)

draw_graph(g)

In [ ]:
import math

def get_height_and_width(g) -> tuple[int, int]:
    height = 0
    width = 0
    for node in g.nodes:
        height = max(int(node.real), height)
        width = max(int(node.imag), width)
    return height, width


def get_num_energized(g, source: complex = -0.5 * 1j):
    s = set()
    height, width = get_height_and_width(g=g)
    for c in nx.descendants(G=g, source=source):
        a = math.ceil(c.real), math.ceil(c.imag)
        if (0 <= a[0] <= height) and (0 <= a[1] <= width):
            s.add(a)

        b = math.floor(c.real), math.floor(c.imag)
        if (0 <= b[0] <= height) and (0 <= b[1] <= width):
            s.add(b)

    return len(s)

In [ ]:
def q_1(data):
    map = parse_data(data)
    g = map_to_graph(map=map)
    return get_num_energized(g=g)


assert q_1(test_data) == 46
# q_1(load_data())
map = parse_data(load_data())
g = map_to_graph(map=map)
# nx.descendants(G=g, source=-0.5 * 1j)
q_1(load_data())

In [ ]:
def q_2(data):
    # just to get the height and width
    map = parse_data(data)
    g = map_to_graph(map=map)
    height, width = get_height_and_width(g=g)

    energy_dict = {}
    for i in range(height + 1):
        source_left_edge = i - 0.5j
        energy_dict[source_left_edge] = get_num_energized(g=g, source=source_left_edge)

        source_right_edge = i + (width + 0.5) * 1j
        energy_dict[source_right_edge] = get_num_energized(g=g, source=source_right_edge)

    for i in range(width + 1):
        source_top_edge = -.5 + i * 1j
        energy_dict[source_top_edge] = get_num_energized(g=g, source=source_top_edge)

        source_bottom_edge = (height + 0.5) + i * 1j
        energy_dict[source_bottom_edge] = get_num_energized(g=g, source=source_bottom_edge)

    return max(energy_dict.values())

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 51
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin